# Assignment 3: Predicting Mapping Penalties with ANN
**Due:** June 5, 2025, 11:59 PM

**Author:** Tony Liang

**Student Number:** 20990204

In this assignment, a feed-forward artificial neural network (ANN) is implemented from scratch to predict the penalty score of a mapping between tasks and employees.

In this notebook we will:
1. Generate or load the 100 mappings dataset  
2. Preprocess & encode into 110-dim vectors  
3. Define two ANN architectures (Model A & Model B)  
4. Implement forward, backward, updates by hand  
5. Train via mini-batch SGD over grid of hyperparameters  
6. Produce the eight required comparison plots  
7. Export results for report submission  



## Assignment Imports

In [74]:
import numpy as np
import pandas as pd
import matplotlib . pyplot as plt
import time

!git clone https://github.com/tonyzrl/ANN_Assignment

# Task data: ID, Estimated Time, Difficulty, Deadline, Skill Required
tasks = [{"id": "T1", "estimated_time": 4, "difficulty": 3, "deadline": 8, "skill_required": "A"},
        {"id": "T2", "estimated_time": 6, "difficulty": 5, "deadline": 12, "skill_required": "B"},
        {"id": "T3", "estimated_time": 2, "difficulty": 2, "deadline": 6, "skill_required": "A"},
        {"id": "T4", "estimated_time": 5, "difficulty": 4, "deadline": 10, "skill_required": "C"},
        {"id": "T5", "estimated_time": 3, "difficulty": 1, "deadline": 7, "skill_required": "A"},
        {"id": "T6", "estimated_time": 8, "difficulty": 6, "deadline": 15, "skill_required": "B"},
        {"id": "T7", "estimated_time": 4, "difficulty": 3, "deadline": 9, "skill_required": "C"},
        {"id": "T8", "estimated_time": 7, "difficulty": 5, "deadline": 14, "skill_required": "B"},
        {"id": "T9", "estimated_time": 2, "difficulty": 2, "deadline": 5, "skill_required": "A"},
        {"id": "T10", "estimated_time": 6, "difficulty": 4, "deadline": 11, "skill_required": "C"},]

# Employee data: ID, Available hours, Skill level, Skills
employees = [{"id": "E1", "hours_avail": 10, "skill_level": 4, "skills": ["A", "C"]},
            {"id": "E2", "hours_avail": 12, "skill_level": 6, "skills": ["A", "B", "C"]},
            {"id": "E3", "hours_avail": 8, "skill_level": 3, "skills": ["A"]},
            {"id": "E4", "hours_avail": 15, "skill_level": 7, "skills": ["B", "C"]},
            {"id": "E5", "hours_avail": 9, "skill_level": 5, "skills": ["A", "C"]}]

fatal: destination path 'ANN_Assignment' already exists and is not an empty directory.


## Data Generation and Loading

In [75]:
df = pd.read_csv('/content/ANN_Assignment/data/task_assignment_data.csv')

## Data Preprocessing

In [76]:
def one_hot_encode(skills):
    """
    One-hot encode a list of skills, e.g. ['A','C'] -> [1,0,1].
    """
    mapping = {'A': 0, 'B': 1, 'C': 2}
    vec = [0, 0, 0]
    for s in skills:
        vec[mapping[s]] = 1
    return vec

def construct_input_vector(mapping_row):
    """
    Given one row of the mapping CSV (task→employee assignments + penalty),
    plus the list of task & employee, construct the 110-dim vector.
    """
    vector = []
    # First 10 entries are employee assignments; last entry is penalty
    assignments = mapping_row[:10]

    for idx, emp_id in enumerate(assignments, start=1):
        task_id = f"T{idx}"
        # Find the task dict
        task = next(t for t in tasks if t["id"] == task_id)
        # Find the employee dict
        emp = next(e for e in employees if e["id"] == emp_id)

        # Task features: [time, difficulty, deadline] + one-hot(required skill)
        task_features = [
            task["estimated_time"],
            task["difficulty"],
            task["deadline"]
        ] + one_hot_encode(task["skill_required"])

        # Employee features: [hours_avail, skill_level] + one-hot(skills)
        emp_features = [
            emp["hours_avail"],
            emp["skill_level"],
        ] + one_hot_encode(emp["skills"])

        vector.extend(task_features + emp_features)

    return np.array(vector)

In [81]:
vector = ["E2","E3","E3","E2","E2","E2","E1","E5","E1","E5",4.6000000000000005]
input = construct_input_vector(vector)
print(input)
print(input.shape)

[ 4  3  8  1  0  0 12  6  1  1  1  6  5 12  0  1  0  8  3  1  0  0  2  2
  6  1  0  0  8  3  1  0  0  5  4 10  0  0  1 12  6  1  1  1  3  1  7  1
  0  0 12  6  1  1  1  8  6 15  0  1  0 12  6  1  1  1  4  3  9  0  0  1
 10  4  1  0  1  7  5 14  0  1  0  9  5  1  0  1  2  2  5  1  0  0 10  4
  1  0  1  6  4 11  0  0  1  9  5  1  0  1]
(110,)


## Activation Functions

In [78]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_derivative(a):
    return a * (1 - a)

def relu(z):
    return np.maximum(0, z)

def relu_derivative(z):
    return (z > 0).astype(float)

## Model A Definitions

In [79]:
class NeuralNetworkA :
  def __init__ (self , layer_dims , activation =("relu")):
  ...
  def forward (self , x):
  ...
  def backward (self , x, y_true ):
  ...
  def update_params (self , lr):
  ...


IndentationError: expected an indented block after function definition on line 2 (<ipython-input-79-10859be4ac2b>, line 3)

## Model B Definitions


In [ ]:
class NeuralNetworkB :
  def __init__ (self , layer_dims , activation =("relu")):
  ...
  def forward (self , x):
  ...
  def backward (self , x, y_true ):
  ...
  def update_params (self , lr):
  ...


## Training Loop